In [1]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64
import random
from pprint import PrettyPrinter

In [2]:
pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/stroke-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1708460880'}]}


In [3]:
pp = PrettyPrinter()
pp.pprint(requests.get("https://strokeprediction-production.up.railway.app/v1/models/stroke-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1708460880'}]}


In [4]:
df = pd.read_csv('data/train_fix.csv')

In [5]:
inputs = df.iloc[11, :10].to_dict()
inputs

{'gender': 'Female',
 'age': 77,
 'hypertension': 1,
 'heart_disease': 0,
 'ever_married': 'Yes',
 'work_type': 'Self-employed',
 'Residence_type': 'Urban',
 'avg_glucose_level': 71.7,
 'bmi': 32.8,
 'smoking_status': 'never smoked'}

In [6]:
df.iloc[53, :11]

gender                       Male
age                            23
hypertension                    0
heart_disease                   0
ever_married                   No
work_type                 Private
Residence_type              Rural
avg_glucose_level           93.74
bmi                          31.2
smoking_status       never smoked
stroke                          0
Name: 53, dtype: object

# Predict

In [7]:
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = tf.train.Feature(float_list=tf.train.FloatList(value=[values]))
        elif isinstance(values, int):
            feature_spec[keys] = tf.train.Feature(int64_list=tf.train.Int64List(value=[values]))
        elif isinstance(values, str):
            feature_spec[keys] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[values.encode()]))

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]

    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

json_data = prepare_json(inputs)

endpoint = "https://strokeprediction-production.up.railway.app/v1/models/stroke-model:predict"
response = requests.post(endpoint, data=json_data)

prediction = response.json().get("predictions")
if prediction:
    prediction_value = prediction[0][0]
    result = "No Stroke" if prediction_value < 0.5 else "Stroke"
else:
    result = "Error: No predictions found in response."

print(result)

No Stroke


In [8]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
